## Downloading GroundingDINO and its wheights

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/home/bruno/prog/tcc


In [3]:
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
%pip install -q -e .
%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights
!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth
%cd {HOME}

fatal: destination path 'GroundingDINO' already exists and is not an empty directory.
/home/bruno/prog/tcc/GroundingDINO


/home/bruno/prog/tcc/env/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Note: you may need to restart the kernel to use updated packages.
/home/bruno/prog/tcc
/home/bruno/prog/tcc/weights
/home/bruno/prog/tcc


## Importing Libraries

In [5]:
import numpy as np
import supervision as sv
from groundingdino.util.inference import load_model, load_image, predict, annotate
import json

In [6]:
CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
print(CONFIG_PATH, "; exist:", os.path.isfile(CONFIG_PATH))
print(WEIGHTS_PATH, "; exist:", os.path.isfile(WEIGHTS_PATH))

model = load_model(CONFIG_PATH, WEIGHTS_PATH)

/home/bruno/prog/tcc/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True
/home/bruno/prog/tcc/weights/groundingdino_swint_ogc.pth ; exist: True


final text_encoder_type: bert-base-uncased


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [ ]:
DATASET_NAME = "val2017"
IMAGES_FOLDER = os.path.join(HOME, "coco_data", DATASET_NAME)
IMAGES_DATA_FILE = os.path.join(HOME, "coco_data", "annotations", f"instances_{DATASET_NAME}.json")

In [7]:
categories = {"person":1,"bicycle":2,"car":3,"motorcycle":4,"airplane":5,"bus":6,"train":7,"truck":8,"boat":9,"traffic light":10,"fire hydrant":11,"stop sign":13,"parking meter":14,"bench":15,"bird":16,"cat":17,"dog":18,"horse":19,"sheep":20,"cow":21,"elephant":22,"bear":23,"zebra":24,"giraffe":25,"backpack":27,"umbrella":28,"handbag":31,"tie":32,"suitcase":33,"frisbee":34,"skis":35,"snowboard":36,"sports ball":37,"kite":38,"baseball bat":39,"baseball glove":40,"skateboard":41,"surfboard":42,"tennis racket":43,"bottle":44,"wine glass":46,"cup":47,"fork":48,"knife":49,"spoon":50,"bowl":51,"banana":52,"apple":53,"sandwich":54,"orange":55,"broccoli":56,"carrot":57,"hot dog":58,"pizza":59,"donut":60,"cake":61,"chair":62,"couch":63,"potted plant":64,"bed":65,"dining table":67,"toilet":70,"tv":72,"laptop":73,"mouse":74,"remote":75,"keyboard":76,"cell phone":77,"microwave":78,"oven":79,"toaster":80,"sink":81,"refrigerator":82,"book":84,"clock":85,"vase":86,"scissors":87,"teddy bear":88,"hair drier":89,"toothbrush":90}

In [8]:
images = json.loads(open(IMAGES_DATA_FILE).read())['images']

In [9]:
from torchvision.ops.boxes import box_convert

def convert_gd_result(img_id, img_width, img_height, boxes, logits, phrases) -> np.ndarray:
    result = []
    boxes = box_convert(boxes, 'cxcywh', 'xywh').numpy()
    logits = logits.numpy()

    for box, logit, phrase in zip(boxes, logits, phrases):
        box[0] = box[0] * img_width
        box[1] = box[1] * img_height
        box[2] = box[2] * img_width
        box[3] = box[3] * img_height
        result.append({
            "image_id": img_id,
            "bbox": box.tolist(),
            "score": logit.item(),
            "category_id": categories[phrase] if phrase in categories else -1,
        })
    return np.array(result)

In [11]:

TEXT_PROMPT = ", ".join(categories.keys())

format = [{"image_id": 0, "bbox": [0, 0, 0, 0], "score": 0.0, "category_id": 0}]

results = np.empty_like(format)

for i, img in enumerate(images):
    if (i+1) % 100 == 0:
        print(f"Processing image {i+1}/{len(images)}")
    IMAGE_PATH = os.path.join(IMAGES_FOLDER, img["file_name"])

    image_source, image = load_image(IMAGE_PATH)

    BOX_TRESHOLD = 0.35
    TEXT_TRESHOLD = 0.25

    boxes, logits, phrases = predict(
        model=model,
        image=image,
        caption=TEXT_PROMPT,
        box_threshold=BOX_TRESHOLD,
        text_threshold=TEXT_TRESHOLD,
    )

    result = convert_gd_result(img["id"], img["width"], img["height"], boxes, logits, phrases)
    results = np.concatenate((results, result))
    # annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)

    # %matplotlib inline
    # sv.plot_image(annotated_frame, (16, 16))

Processing image 100/5000


In [ ]:
with open(f"gd_results_{DATASET_NAME}.json", "w") as f:
    json.dump(results.tolist(), f)